In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from google.colab import drive

drive.mount('/content/drive')

base_dir = "/content/drive/MyDrive/BTECHPROJ"
img_size = (224, 224)
batch_size = 16
num_classes = 3
epochs = 40
learning_rate = 2e-4  # Slightly higher for stronger warmup

# Moderate augmentation for bacteria images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=18,
    width_shift_range=0.08,
    height_shift_range=0.08,
    brightness_range=[0.9, 1.12],
    shear_range=0.09,
    zoom_range=0.13,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='categorical',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical',
    shuffle=False,
    seed=42
)

# ResNet50V2 base model
base_model = ResNet50V2(weights="imagenet", include_top=False, input_shape=img_size + (3,))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Dense(384, activation='relu')(x)       # Slightly larger dense for more representation
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    ModelCheckpoint(
        filepath="/content/drive/MyDrive/BTECHPROJ/best_model.keras",
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=7,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=2,
        min_lr=1e-6,
        verbose=1
    )
]

# Warmup: frozen base training
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

# Fine-tuning: unfreeze deeper layers (last 90 layers)
base_model.trainable = True
for layer in base_model.layers[:-90]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 20
total_epochs = epochs + fine_tune_epochs

history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=callbacks,
    verbose=1
)

# Restore best weights for evaluation
model.load_weights('/content/drive/MyDrive/BTECHPROJ/best_model.keras')
val_loss, val_acc = model.evaluate(validation_generator, verbose=0)
print("Training complete.")
print(f"Best restored validation accuracy: {val_acc*100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 254 images belonging to 3 classes.
Found 63 images belonging to 3 classes.
Epoch 1/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4605 - loss: 1.4006
Epoch 1: val_accuracy improved from -inf to 0.66667, saving model to /content/drive/MyDrive/BTECHPROJ/best_model.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 90s 5s/step - accuracy: 0.4668 - loss: 1.3818 - val_accuracy: 0.6667 - val_loss: 0.7338 - learning_rate: 2.0000e-04
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6900 - loss: 0.8116
Epoch 2: val_accuracy improved from 0.66667 to 0.69841, saving model to /content/drive/MyDrive/BTECHPROJ/best_model.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 72s 4s/step - accuracy: 0.6907 - loss: 0.8081 - val_accuracy: 0.6984 - val_loss: 0.6142 - learning_rate: 2.0000e-04
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7543 - loss: 0.5284
Epoch 3: val_a

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 182 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Training complete.
Best restored validation accuracy: 80.95%


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2, DenseNet121
from tensorflow.keras.layers import (
    Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Concatenate, Input
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
)
from tensorflow.keras.losses import CategoricalCrossentropy
from google.colab import drive

drive.mount('/content/drive')

# Paths & hyperparameters
base_dir = "/content/drive/MyDrive/BTECHPROJ"
img_size = (224, 224)
batch_size = 16
num_classes = 3
epochs = 40

# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='categorical',
    shuffle=True,
    seed=42
)
val_gen = val_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical',
    shuffle=False,
    seed=42
)

# --- Load your ResNet model base ---
resnet_base = ResNet50V2(weights="imagenet", include_top=False, input_shape=img_size + (3,))
resnet_base.trainable = False

# --- Load DenseNet base ---
densenet_base = DenseNet121(weights="imagenet", include_top=False, input_shape=img_size + (3,))
densenet_base.trainable = False

# Common input
inputs = Input(shape=img_size + (3,))

# Extract features correctly
resnet_features = resnet_base(inputs)
densenet_features = densenet_base(inputs)

# Feature Fusion (same spatial size: 7x7)
merged = Concatenate(axis=-1)([resnet_features, densenet_features])  # 7x7x(3072)

x = GlobalAveragePooling2D()(merged)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)

hybrid_model = Model(inputs=inputs, outputs=outputs)

# Compile
hybrid_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    ModelCheckpoint(
        filepath="/content/drive/MyDrive/BTECHPROJ/best_hybrid_fixed.keras",
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

# Train the hybrid model
print("Training properly fused ResNet50V2 + DenseNet121 feature ensemble...")
history = hybrid_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

# Evaluate final model
hybrid_model.load_weights("/content/drive/MyDrive/BTECHPROJ/best_hybrid_fixed.keras")
val_loss, val_acc = hybrid_model.evaluate(val_gen, verbose=0)
print(f"Hybrid ensemble training complete. Final validation accuracy: {val_acc * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 254 images belonging to 3 classes.
Found 63 images belonging to 3 classes.
94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training properly fused ResNet50V2 + DenseNet121 feature ensemble...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.3961 - loss: 1.6159 
Epoch 1: val_accuracy improved from -inf to 0.76190, saving model to /content/drive/MyDrive/BTECHPROJ/best_hybrid_fixed.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 322s 19s/step - accuracy: 0.4029 - loss: 1.6003 - val_accuracy: 0.7619 - val_loss: 0.7321 - learning_rate: 1.0000e-04
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.6679 - loss: 0.9578
Epoch 2: val_accuracy did not improve from 0.76190
16/16 ━━━━━━━━━━━━━━━━━━━━ 117s 7s/step - accuracy: 0.6710 - loss: 0.9541 - val_accuracy: 0.7619 - val_loss: 0.6848 - learning_rate: 1.0000e-04
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7817 - loss: 0.7365
Epoch 3: val_accuracy improved from 0.76190 to 0.77778, saving model to /content/drive/MyDrive/BTECHPROJ/best_hybrid_fixed.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 143s 7s/step - accuracy: 0.7804 - loss: 0.7388 - val_accuracy: 0.7778 - val_loss: 0.6631 - learning_rate: 1.0000e-04
Epoch 4/40

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2, DenseNet121, MobileNetV2
from tensorflow.keras.layers import (
    Dense, GlobalAveragePooling2D, Dropout, BatchNormalization,
    Concatenate, Input, Multiply, Add
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
)
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from google.colab import drive

drive.mount('/content/drive')

# ====================
# Paths & Hyperparams
# ====================
base_dir = "/content/drive/MyDrive/BTECHPROJ"
img_size = (224, 224)
batch_size = 16
num_classes = 3
epochs = 50
initial_lr = 1e-4

# ================
# Data Generators
# ================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    brightness_range=[0.8, 1.2],
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='reflect'
)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = train_datagen.flow_from_directory(
    base_dir, target_size=img_size, batch_size=batch_size,
    subset='training', class_mode='categorical', shuffle=True, seed=42
)
val_gen = val_datagen.flow_from_directory(
    base_dir, target_size=img_size, batch_size=batch_size,
    subset='validation', class_mode='categorical', shuffle=False, seed=42
)

# ================
#  Backbone Models
# ================
resnet = ResNet50V2(weights="imagenet", include_top=False, input_shape=img_size + (3,))
densenet = DenseNet121(weights="imagenet", include_top=False, input_shape=img_size + (3,))
mobilenet = MobileNetV2(weights="imagenet", include_top=False, input_shape=img_size + (3,))

resnet.trainable = False
densenet.trainable = False
mobilenet.trainable = False

# ==========================
# Attention‑weighted Fusion
# ==========================
inputs = Input(shape=img_size + (3,))

r_feat = resnet(inputs)
d_feat = densenet(inputs)
m_feat = mobilenet(inputs)

# All outputs are roughly (7×7×n); concatenate feature maps
merged = Concatenate(axis=-1)([r_feat, d_feat, m_feat])

from tensorflow.keras.layers import Reshape, Multiply

gap = GlobalAveragePooling2D()(merged)
gate = Dense(merged.shape[-1], activation="sigmoid")(gap)
gate_reshaped = Reshape((1, 1, merged.shape[-1]))(gate)
weighted_features = Multiply()([merged, gate_reshaped])  # apply attention

# =====================
#  Classification Head
# =====================
x = GlobalAveragePooling2D()(weighted_features)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# ===================
#  Compile & Metrics
# ===================
model.compile(
    optimizer=Adam(learning_rate=initial_lr),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# ===============
#  Callback setup
# ===============
callbacks = [
    ModelCheckpoint(
        filepath="/content/drive/MyDrive/BTECHPROJ/best_tri_hybrid.keras",
        monitor='val_accuracy', save_best_only=True,
        mode='max', verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy', patience=8,
        restore_best_weights=True, verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss', factor=0.3,
        patience=3, min_lr=1e-7, verbose=1
    )
]

# ==================
#  Train the model
# ==================
print("Training triple hybrid model (ResNet + DenseNet + MobileNet with attention fusion)...")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

# =======================
#  Evaluation & Results
# =======================
model.load_weights("/content/drive/MyDrive/BTECHPROJ/best_tri_hybrid.keras")
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"Triple‑hybrid model complete. Final validation accuracy: {val_acc * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 254 images belonging to 3 classes.
Found 63 images belonging to 3 classes.
Training triple hybrid model (ResNet + DenseNet + MobileNet with attention fusion)...
Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.4905 - loss: 2.2656
Epoch 1: val_accuracy improved from -inf to 0.58730, saving model to /content/drive/MyDrive/BTECHPROJ/best_tri_hybrid.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 175s 9s/step - accuracy: 0.5006 - loss: 2.2473 - val_accuracy: 0.5873 - val_loss: 1.7272 - learning_rate: 1.0000e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8843 - loss: 1.5111
Epoch 2: val_accuracy improved from 0.58730 to 0.71429, saving model to /content/drive/MyDrive/BTECHPROJ/best_tri_hybrid.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 155s 10s/step - accuracy: 0.8846 - loss: 1.5111 - val_accuracy: 0.7143 - val_loss: 1.6400 - learning_rate: 1.0000e-0

In [ ]:
#iske neechey wala hai main code.

In [ ]:
import math
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2, DenseNet121, MobileNetV2
from tensorflow.keras.layers import (
    Dense, GlobalAveragePooling2D, Dropout, BatchNormalization,
    Concatenate, Input, Multiply, Reshape
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
)
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# Parameters
base_dir = "/content/drive/MyDrive/BTECHPROJ"
img_size = (224, 224)
batch_size = 16
num_classes = 3
epochs_initial = 40
epochs_finetune = 30
initial_lr = 1e-4

# Data Augmentation and Validation Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    brightness_range=[0.8, 1.2],
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='reflect'
)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='categorical',
    shuffle=True,
    seed=42
)
val_gen = val_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical',
    shuffle=False,
    seed=42
)

# Load backbone models
resnet = ResNet50V2(weights="imagenet", include_top=False, input_shape=img_size + (3,))
densenet = DenseNet121(weights="imagenet", include_top=False, input_shape=img_size + (3,))
mobilenet = MobileNetV2(weights="imagenet", include_top=False, input_shape=img_size + (3,))

# Freeze backbones initially
resnet.trainable = False
densenet.trainable = False
mobilenet.trainable = False

# Input and extract features
inputs = Input(shape=img_size + (3,))
r_feat = resnet(inputs)
d_feat = densenet(inputs)
m_feat = mobilenet(inputs)

merged = Concatenate(axis=-1)([r_feat, d_feat, m_feat])

gap = GlobalAveragePooling2D()(merged)
gate = Dense(merged.shape[-1], activation="sigmoid")(gap)
gate_reshaped = Reshape((1, 1, merged.shape[-1]))(gate)
weighted_features = Multiply()([merged, gate_reshaped])

# Classification head
x = GlobalAveragePooling2D()(weighted_features)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=initial_lr),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# Callbacks including cosine annealing LR scheduler
def cosine_annealing_scheduler(epoch, lr):
    max_lr = initial_lr
    min_lr = 1e-7
    decay = 0.5 * (1 + math.cos(math.pi * epoch / epochs_finetune))
    new_lr = min_lr + (max_lr - min_lr) * decay
    print(f"Epoch {epoch+1}: setting learning rate to {new_lr}")
    return new_lr

callbacks = [
    ModelCheckpoint(
        filepath="/content/drive/MyDrive/BTECHPROJ/best_tri_hybrid_cosine.keras",
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    LearningRateScheduler(cosine_annealing_scheduler, verbose=1)
]

# Train frozen backbones first
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs_initial,
    callbacks=callbacks,
    verbose=1
)

# Unfreeze last layers progressively for fine-tuning
backbones = [resnet, densenet, mobilenet]
for backbone in backbones:
    # Freeze all layers except the last 30
    for layer in backbone.layers[:-30]:
        layer.trainable = False
    for layer in backbone.layers[-30:]:
        layer.trainable = True

# Compile fine-tuning model with low LR
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# Fine-tune
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs_initial + epochs_finetune,
    initial_epoch=history.epoch[-1],
    callbacks=callbacks,
    verbose=1
)

# Load best weights and evaluate
model.load_weights("/content/drive/MyDrive/BTECHPROJ/best_tri_hybrid_cosine.keras")
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"Tri-model hybrid with cosine LR training complete. Validation accuracy: {val_acc*100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 254 images belonging to 3 classes.
Found 63 images belonging to 3 classes.
Epoch 1: setting learning rate to 0.0001

Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.4523 - loss: 2.3520
Epoch 1: val_accuracy improved from -inf to 0.53968, saving model to /content/drive/MyDrive/BTECHPROJ/best_tri_hybrid_cosine.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 181s 9s/step - accuracy: 0.4620 - loss: 2.3303 - val_accuracy: 0.5397 - val_loss: 1.8141 - learning_rate: 1.0000e-04
Epoch 2: setting learning rate to 9.972636867364526e-05

Epoch 2: LearningRateScheduler setting learning rate to 9.972636867364526e-05.
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8122 - loss: 1.6007
Epoch 2: val_accuracy improved from 0.53968 to 0.69841, saving model to /content/drive/MyDrive/BTECHPROJ/be